In [ ]:
#tools.py
import nacl.public, nacl.encoding
import os
import stat
import gzip

def assert_permissions(name, permission):
    message = f"Permission of '{name}' must be '{permission}'. "\
               "See 'chmod' for help"
    assert stat.filemode(os.stat(name).st_mode)[1:] == permission, message

def create_keys(fullname):
    ''' Create secret and public keys for nacl
    
        The keys are stored in path in hexencoding where
        path,name = os.split(fullname)
        
        If path does not exist, raise an error
        If path has wrong permissions, raise an error:
        Only onwner can read,write or execute path (i.e. chmod og-rwx u=rwx path)
        
        If fullname exists, raise an error
        
        The permissions of the resulting secret keyfile is og-rwx
        (Only owner can read,write or execute secret keyfile)
    '''
    
    path,name = os.path.split(fullname)
    pub_name = fullname+'.pub'
    
    # path must exist with permissions 'rwx------'
    assert os.path.exists(path), f"Path '{path}' does not exist"
    assert_permissions(path,'rwx------')
    
    # fullname must not exist
    assert not os.path.exists(fullname), f"keyfile: '{fullname}' exists already"
    
    with open(fullname,'w') as s_f, open(pub_name,'w') as p_f:
        key = nacl.public.PrivateKey.generate()
        
        s_f.write(key.encode(nacl.encoding.HexEncoder).decode())
        p_f.write(key.public_key.encode(nacl.encoding.HexEncoder).decode())
        
    os.chmod(fullname,0o600)

def read_secret_key(fullname):
    ''' Read a private key for nacl from fullname.
        The path and the fullname must have the right permissions, og-rwx
        (Only user can read,write or execute secret key)
    '''
    
    # fullpath must exist with permissions 'rw-------'
    assert os.path.exists(fullpath), f"File '{fullpath}' does not exist"
    assert_permissions(fullpath,'rw-------')
        
    path,name = os.path.split(fullname)
    
    # path must exist with permissions 'rwx------'
    assert_permissions(path,'rwx------')
    
    with open(fullname,'r') as f:
        key = nacl.public.PrivateKey(f.read().encode(),encoder=nacl.encoding.HexEncoder)
        
    return key

def read_public_key(fullname):
    ''' Read a public key for nacl from fullname'''
    
    with open(fullname,'r') as f:
        key = nacl.public.PublicKey(f.read().encode(),encoder=nacl.encoding.HexEncoder)
    
    return key

def encrypt(sk,pk,f_in_name,f_out_name):
    ''' encrypt f_in_name to f_out_name using nacl with 
        secret key sk and public key pk. If f_out_name
        ends with '.gz', use gzip
    '''
    gzipped = os.path.splitext(f_out_name)[1] == '.gz'
    out = gzip.open if gzipped else open
    
    with open(f_in_name,'rb') as f_in, out(f_out_name,'wb') as f_out:
        f_out.write(nacl.public.Box(sk,pk).encrypt(f_in.read()))


def decrypt(sk,pk,f_in_name,f_out_name):
    ''' decrypt f_in_name to f_out_name using nacl with
        secret key sk und public key pk. If f_in_name ends
        with '.gz' use gzip
    '''
    gzipped = os.path.splitext(f_in_name)[1] == '.gz'
    in_ = gzip.open if gzipped else open
    with in_(f_in_name,'rb') as f_in, open(f_out_name,'wb') as f_out:
        f_out.write(nacl.public.Box(sk,pk).decrypt(f_in.read()))


In [1]:
from tools import *

In [2]:
!rm -rf ./.encr/

In [3]:
!ls -hal

total 96
drwxr-xr-x  14 wolfgang  wheel   448B 10 Jul 14:01 .
drwxr-xr-x   3 wolfgang  wheel    96B 10 Jul 13:03 ..
drwxr-xr-x  12 wolfgang  wheel   384B 10 Jul 13:03 .git
-rw-r--r--   1 wolfgang  wheel    37B 10 Jul 13:03 .gitignore
drwxr-xr-x   3 wolfgang  wheel    96B 10 Jul 13:09 .ipynb_checkpoints
-rw-r--r--   1 wolfgang  wheel   6,5K 10 Jul 13:03 Datei_verschluesseln_und_entschluesseln.ipynb
-rw-r--r--   1 wolfgang  wheel   600B 10 Jul 13:03 README.md
drwxr-xr-x   3 wolfgang  wheel    96B 10 Jul 14:01 __pycache__
drwxr-xr-x   5 wolfgang  wheel   160B 10 Jul 14:04 alice
drwxr-xr-x   5 wolfgang  wheel   160B 10 Jul 14:04 bob
-rw-r--r--   1 wolfgang  wheel    16K 10 Jul 13:35 test_tools.ipynb
-rw-r--r--   1 wolfgang  wheel    18B 10 Jul 13:03 text.txt
-rw-r--r--   1 wolfgang  wheel   3,7K 10 Jul 13:03 tools.ipynb
-rw-r--r--   1 wolfgang  wheel   3,2K 10 Jul 14:01 tools.py


In [4]:
create_keys('./.encr/secret_key')

AssertionError: Path './.encr' does not exist

In [5]:
!mkdir ./.encr
create_keys('./.encr/secret_key')

AssertionError: Permission of './.encr' must be 'rwx------'. See 'chmod' for help

In [6]:
!chmod og-rwx ./.encr
create_keys('./.encr/secret_key')

In [7]:
create_keys('./.encr/secret_key')

AssertionError: keyfile: './.encr/secret_key' exists already

In [8]:
!ls -hla . ./.encr

.:
total 96
drwxr-xr-x  15 wolfgang  wheel   480B 10 Jul 14:05 .
drwxr-xr-x   3 wolfgang  wheel    96B 10 Jul 13:03 ..
drwx------   4 wolfgang  wheel   128B 10 Jul 14:05 .encr
drwxr-xr-x  12 wolfgang  wheel   384B 10 Jul 13:03 .git
-rw-r--r--   1 wolfgang  wheel    37B 10 Jul 13:03 .gitignore
drwxr-xr-x   3 wolfgang  wheel    96B 10 Jul 13:09 .ipynb_checkpoints
-rw-r--r--   1 wolfgang  wheel   6,5K 10 Jul 13:03 Datei_verschluesseln_und_entschluesseln.ipynb
-rw-r--r--   1 wolfgang  wheel   600B 10 Jul 13:03 README.md
drwxr-xr-x   3 wolfgang  wheel    96B 10 Jul 14:01 __pycache__
drwxr-xr-x   5 wolfgang  wheel   160B 10 Jul 14:04 alice
drwxr-xr-x   5 wolfgang  wheel   160B 10 Jul 14:04 bob
-rw-r--r--   1 wolfgang  wheel    16K 10 Jul 13:35 test_tools.ipynb
-rw-r--r--   1 wolfgang  wheel    18B 10 Jul 13:03 text.txt
-rw-r--r--   1 wolfgang  wheel   3,7K 10 Jul 13:03 tools.ipynb
-rw-r--r--   1 wolfgang  wheel   3,2K 10 Jul 14:01 tools.py

./.encr:
total 16
drwx------   4 wolfgang  wheel   